In [2]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics
import seaborn as sns; sns.set(style="whitegrid")

%matplotlib inline

In [3]:
df = pd.read_csv('datasets/data_cart.csv', index_col=0)#[['ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3']]
pos = df.shape[0]-160
#del df['Data']
y_symbol = 'CMIG4'
x_symbol = 'ENBR3'

S1 = df[x_symbol][pos:]
S2 = df[y_symbol][pos:]

In [21]:
prices = df['AZUL4']

In [59]:
def volatility(prices, periods=0):
    if (periods > 0):
        prices = prices[0:periods+1]# +1, pq haverá um lag(d-1)
    log = np.log(prices/prices.shift(1))
    vol = log.std() * np.sqrt(252)
    return vol

In [56]:
volatility(prices)

Data
08/04/2019         NaN
09/04/2019   -0.008357
10/04/2019    0.008357
11/04/2019   -0.035864
12/04/2019   -0.030062
                ...   
16/06/2020    0.019670
17/06/2020    0.001327
18/06/2020   -0.030979
19/06/2020   -0.005946
22/06/2020   -0.055164
Name: AZUL4, Length: 300, dtype: float64

In [57]:
volatility(prices, 21)

Data
08/04/2019         NaN
09/04/2019   -0.008357
10/04/2019    0.008357
11/04/2019   -0.035864
12/04/2019   -0.030062
15/04/2019   -0.031605
16/04/2019    0.014572
17/04/2019   -0.045631
18/04/2019    0.059102
22/04/2019    0.010354
23/04/2019    0.005576
24/04/2019   -0.020400
25/04/2019   -0.006893
26/04/2019    0.010769
29/04/2019    0.001487
30/04/2019    0.010050
02/05/2019    0.017493
03/05/2019    0.041885
06/05/2019   -0.030391
07/05/2019    0.007968
08/05/2019    0.047602
09/05/2019    0.026668
Name: AZUL4, dtype: float64

In [58]:
volatility(prices, 30)

Data
08/04/2019         NaN
09/04/2019   -0.008357
10/04/2019    0.008357
11/04/2019   -0.035864
12/04/2019   -0.030062
15/04/2019   -0.031605
16/04/2019    0.014572
17/04/2019   -0.045631
18/04/2019    0.059102
22/04/2019    0.010354
23/04/2019    0.005576
24/04/2019   -0.020400
25/04/2019   -0.006893
26/04/2019    0.010769
29/04/2019    0.001487
30/04/2019    0.010050
02/05/2019    0.017493
03/05/2019    0.041885
06/05/2019   -0.030391
07/05/2019    0.007968
08/05/2019    0.047602
09/05/2019    0.026668
10/05/2019    0.002366
13/05/2019   -0.039904
14/05/2019   -0.040424
15/05/2019   -0.014904
16/05/2019   -0.039163
17/05/2019   -0.019406
20/05/2019    0.022704
21/05/2019    0.053327
22/05/2019    0.012409
Name: AZUL4, dtype: float64